In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from Data_handler import Data, RF_COL
from termcolor import colored

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os

# 2 Loading the data

In [ ]:
# Prepare data
Data_instance = Data()               # create instance of the class Data()
data = Data_instance.get_data()      # store the data of Data_instance

In [ ]:
data.sort_values(by=['permno','date'],inplace=True)
print(data.shape)

# 3 Betting Against Beta

In [ ]:
data.groupby('date').first().sort_values(by='date')

## Question a)

In [ ]:
# Remove stocks with less than 36 months of observation (code from PS5)
# data['N'] = data.groupby(['permno'])['date'].transform('count')
# data = data[data['N']>36].copy()

# Add excess returns
data['Rn_e'] = data['ret'] - data[RF_COL]
data['Rm_e'] = data['vwretd'] - data[RF_COL]

# Drop nan values
data = data.dropna().copy()

# Rolling betas for each stock, based on 5-year windows (code from PS5)
w = 5 * 12  # window size
covariance = data.set_index('date').groupby('permno')[['Rn_e', 'Rm_e']].rolling(window=w, min_periods=36).cov()
betas = covariance.iloc[1::2,1].droplevel(2) / covariance.iloc[0::2,1].droplevel(2)
betas = betas.dropna().reset_index().rename(columns={'Rm_e': 'beta'})

# Make sure the dates columns are datetime
betas.date = pd.to_datetime(betas.date)
data.date = pd.to_datetime(data.date)

# Offset the dates of the betas by 1 month (code from PS5)
betas.date = betas.date + pd.DateOffset(months=1)

# Merge the full data with betas (code from PS5)
data_Qb = pd.merge(data, betas, on=['permno', 'date'], how='left')

# Finally, we winsorize the betas (5% and 95%) (code from PS5)
data_Qb['beta'] = data_Qb['beta'].clip(data_Qb['beta'].quantile(0.05), data_Qb['beta'].quantile(0.95))

# Drop all nan
data_Qb = data_Qb.dropna().copy()

print('Distribution of betas:')
print(data_Qb['beta'].describe(), '\n')


display(data_Qb.head()) # Overview of the data
print(data_Qb.shape)

In [ ]:
# data_Qb = data.copy().dropna() # Uncomment if decide to use the class method to get the rolling betas, which gives different results 😃
print("Initial number of observations: ", data.shape[0])
print("Final number of observations: \t", data_Qb.shape[0])

## Question b)

### Equally weighted portfolios

In [ ]:
# Create deciles based on Beta value (code from PS5)
data_Qb["EW_monthly_decile"] = data_Qb.groupby("date")["beta"].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))

# Equally weighted returns per month, for each decile
EW_returns = data_Qb.groupby(["date", "EW_monthly_decile"]).agg({
    'ret': 'mean',
    RF_COL: 'first',
    'EW_monthly_decile': 'first',
    'date': 'first'
    }).reset_index(drop=True).rename(columns={'EW_monthly_decile': 'decile'})

print("Equally weighted returns per month, for each decile:")
display(EW_returns.head(5))

In [ ]:
# Value weighted returns per month, for each decile
data_Qb['VW_weight'] = data_Qb.groupby(['date', 'EW_monthly_decile'])['mcap'].transform(lambda x: x / x.sum())
data_Qb['VW_ret_contrib'] = data_Qb['VW_weight'] * data_Qb['ret']

VW_returns = data_Qb.groupby(["date", "EW_monthly_decile"]).agg({
    'VW_ret_contrib': 'sum',
    RF_COL: 'first',
    'EW_monthly_decile': 'first',
    'date': 'first'
    }).reset_index(drop=True).rename(columns={'EW_monthly_decile': 'decile', 'VW_ret_contrib': 'ret'})

print("Value weighted returns per month, for each decile:")
display(VW_returns.head(5))

### Plot the mean returns, volatility, and sharpe ratios for EW and VW portfolios

#### Auxiliary functions for plots

In [ ]:
def get_mean_std_sharpe(data):
    """Compute the annulalized mean, standard deviation and Sharpe ratio for each decile."""
    mean = list(map(lambda x: 12*x, data.groupby('decile')['ret'].mean().values.tolist()))
    std = list(map(lambda x: np.sqrt(12)*x, data.groupby('decile')['ret'].std().values.tolist()))
    rf = np.mean(list(map(lambda x: 12*x, data.groupby('decile')[RF_COL].mean().values.tolist())))
    sr = list(map(lambda ret, vol: (ret - rf) / vol, mean, std))
    
    return mean, std, sr

def plot_from_lists(mean, std, sharpe, plot_color = 'blue'):
    deciles = list(range(len(mean)))

    _, axs = plt.subplots(1, 3, figsize=(25, 7), sharey=False)

    axs[0].bar(deciles, mean, color=plot_color)
    axs[0].set_title("Average portolio mean return")
    axs[0].set_xticks(deciles)
    axs[0].set_xlabel("Decile")
    axs[0].set_ylabel("Annualized return")

    axs[1].bar(deciles, std, color=plot_color)
    axs[1].set_title("Average portolio annualized standard deviation")
    axs[1].set_xticks(deciles)
    axs[1].set_xlabel("Decile")
    axs[1].set_ylabel("Annualized standard deviation")

    axs[2].bar(deciles, sharpe, color=plot_color)
    axs[2].set_title("Average portolio annualized sharpe ratio")
    axs[2].set_xticks(deciles)
    axs[2].set_xlabel("Decile")
    axs[2].set_ylabel("Annualized sharpe ratio")
    
    return plt

def plot_mean_std_sr(data, question, plot_name, show = True):
    """Takes a dataframe in input and returns a 3 graphs for the  annualized mean, std and sharpe ratio for some deciles."""

    if not os.path.exists("Figures"):
            os.makedirs("Figures")
    
    mean, std, sharpe = get_mean_std_sharpe(data)

    plot = plot_from_lists(mean, std, sharpe, plot_color = 'blue')

    plot.suptitle(f'Average portolio annualized mean return, standard deviation and sharpe ratio ({plot_name})')
    plot.savefig(f"Figures/question_{question}_plot_{plot_name}")

    if show: 
        plot.show()

#### Plots

In [ ]:
# Plot the results for the 2 different weightings
plot_mean_std_sr(EW_returns, '3b', "EW_returns_bab")
plot_mean_std_sr(VW_returns, '3b', "VW_returns_bab")

## Question c) and d)

### Built the function for determining BAB weights

In [ ]:
def get_bab_weights(data):
    """Computes the weights of the Betting-Against-Beta portfolio (code inspired from PS5)."""
    df = data.copy()
    df['z'] = df.groupby('date')['beta'].rank()                     # Assign each beta a rank, for each month
    df['z_mean'] = df.groupby('date')['z'].transform('mean')        # Calculate the monthly mean the rank
    df['norm'] = np.abs(df['z']- df['z_mean'])                      # Compute abs distance of rank to mean rank
    df['sum_norm'] = df.groupby('date')['norm'].transform("sum")    # Sum the distance
    df['k'] = 2 / df['sum_norm']                                    # Compute the k

    # Compute the BAB weights
    df['wH'] = df['k'] * np.maximum(0, df['z'] - df['z_mean'])
    df['wL'] = - df['k'] * np.minimum(0, df['z'] - df['z_mean'])

    # Drop irrelevant columns
    df = df.drop(columns=["z_mean", 'z', 'norm', 'sum_norm', 'k'])

    # Compute the weighted betas
    df['bH'] = df['wH'] * df['beta']
    df['bL'] = df['wL'] * df['beta']

    # Compute the individual excess returns of the portfolios H and L
    df['rH_e'] = df['wH'] * (df['ret'] - df[RF_COL])
    df['rL_e'] = df['wL'] * (df['ret'] - df[RF_COL]) # Check that crazy formula bby 😃  (en gros, c'est okay de faire weight * excess return au lieu de faire weight * excess return?)
    
    # Compute the return and betas of the two portfolios for each period
    df_ = df.groupby('date').agg({
        'rH_e': 'sum',
        'rL_e': 'sum',
        'bH': 'sum',
        'bL': 'sum',
        'Rm_e': 'first',
    }).reset_index()

    # Finally create the BAB portfolio return
    df_['rBAB'] = df_['rL_e'] / df_['bL'] - df_['rH_e'] / df_['bH']

    return df_

In [ ]:
# Create the weights rBAB
data_BAB = get_bab_weights(data_Qb)
display(data_BAB)

### Get the return, std and sharpe ratio of the BAB strategy

In [ ]:
# We compute the rf based on question b) results, as the underlying data is the same
rf = np.mean(list(map(lambda x: 12*x, VW_returns.groupby('decile')[RF_COL].mean().values.tolist())))

# Compute the return, std and Sharpe ratio of the BAB strategy
BAB_ret = data_BAB.rBAB.mean() * 12
BAB_std = data_BAB.rBAB.std() * np.sqrt(12)
BAB_shr = (BAB_ret - rf) / BAB_std

# Compute the CAPM alpha
data_BAB['one'] = 1 # Create the column for the constant
model = sm.OLS(data_BAB['rBAB'], data_BAB[['one', 'Rm_e']]).fit() # Fit CAPM

print(colored("Betting-against-beta strategy", "black", attrs=['underline', 'bold']))
print(" - Mean return: {:.2f}".format(BAB_ret))
print(" - Standard deviation: {:.2f}".format(BAB_std))
print(" - Sharpe ratio: {:.2f}".format(BAB_shr))
print(" - CAPM alpha: {:.2f}".format(model.params.iloc[0] * 12))

# 4 Momentum Strategy

## Question a)

### Create the deciles based on the 12-month cumulative return, excluding short term reversal

In [ ]:
data.sort_values(by=['permno', 'date'], inplace=True)
display(data.head())

In [ ]:
# Sort data by permno, then date
data_mom = data.copy()
data_mom.sort_values(by=['permno', 'date'], inplace=True)

# Add a column for momentum return (last 12 months, excluding last month)
data_mom['roll_ret'] = data_mom.groupby('permno').ret.transform(lambda x: x.rolling(11, closed='left').sum())
display(data_mom.head())

In [ ]:
# Create deciles for the momentum returns
data_mom['decile_mom'] = data_mom.groupby('date')['roll_ret'].transform(lambda x: pd.qcut(x, 10, labels=False, duplicates='drop'))

# Compute the monthly return for each decile (this is the average of the individual monthly return of each stock from each decile)
data_mom['EW_monthly_return'] = data_mom.groupby(['date', 'decile_mom'])['ret'].transform('mean')

# Drop nan values
data_mom = data_mom.dropna().copy()

display(data_mom.head())
print(data_mom.shape)
# data_mom.to_csv('data_mom.csv')

### Equally weighted portfolios

In [ ]:
# Equally weighted returns per month, for each decile
EW_returns_mom = data_mom.groupby(['date', 'decile_mom']).agg({
    'ret': 'mean',
    RF_COL: 'first',
    'decile_mom': 'first',
    'date': 'first'
    }).reset_index(drop=True).rename(columns={'decile_mom': 'decile'})

print(colored("Equally weighted returns per month, for each decile:", "black", attrs=['underline', 'bold']))
display(EW_returns_mom.head(5))
print(EW_returns_mom.shape)

### Value weighted portfolios

In [ ]:
# Value weighted returns per month, for each decile
data_mom['VW_weight'] = data_mom.groupby(['date', 'decile_mom'])['mcap'].transform(lambda x: x / x.sum())
data_mom['VW_ret_contrib'] = data_mom['VW_weight'] * data_mom['ret']

VW_returns_mom = data_mom.groupby(["date", "decile_mom"]).agg({
    'VW_ret_contrib': 'sum',
    RF_COL: 'first',
    'decile_mom': 'first',
    'date': 'first'
    }).reset_index(drop=True).rename(columns={'decile_mom': 'decile', 'VW_ret_contrib': 'ret'})

print(colored("Value weighted returns per month, for each decile:", "black", attrs=['underline', 'bold']))
display(VW_returns_mom.head(5))
print(VW_returns_mom.shape)

### Plotting the graphs

In [ ]:
# Plot the results for the 2 different weightings
plot_mean_std_sr(EW_returns_mom, '4a', "EW_returns_mom")
plot_mean_std_sr(VW_returns_mom, '4a', "VW_returns_mom")

## Question b)

### Prepare data

In [ ]:
# Create a column 'leg' that is 1 if the decile is 7, 8 or 9, and -1 if decile is 0, 1, 2
data_mom['leg'] = np.nan
data_mom.loc[data_mom['decile_mom'] <= 2, 'leg'] = -1
data_mom.loc[data_mom['decile_mom'] >= 7, 'leg'] = 1

# Drop the observations that are in none of the legs
data_mom_b = data_mom.dropna().copy()
display(data_mom_b)

### Equally weighted portfolios

In [ ]:
# Create a dataframe that aggregates takes the average return for each leg, at each month. Also keep the risk free rate
EW_data_mom = data_mom_b.groupby(['date', 'leg']).agg({
    'ret': 'mean', 
    RF_COL: 'first',
    }).reset_index()

EW_data_mom_piv = EW_data_mom.pivot(index='date', columns='leg', values='ret') # Pivot the data
EW_data_mom_piv['EW_return'] = EW_data_mom_piv[1] - EW_data_mom_piv[-1] # Compute the return of the EW momentum strategy as being the difference between the two legs
EW_data_mom_piv[RF_COL] = EW_data_mom.groupby('date')[RF_COL].first() # Add the risk free rate
EW_data_mom_piv = EW_data_mom_piv[['EW_return', RF_COL]]   # Keep only the relevant columns
# display(EW_data_mom_piv)

# Compute mean, std and Sharpe ratio
mean = EW_data_mom_piv['EW_return'].mean() * 12
std = EW_data_mom_piv['EW_return'].std() * np.sqrt(12)
rf = EW_data_mom_piv[RF_COL].mean() * 12

# Dispay the results
print(colored("Momentum strategy based on equally weighted portfolios", "black", attrs=['underline', 'bold']))
print(" - Expected return:\t {:.2f}%".format(mean))
print(" - Standard deviation:\t {:.2f}%".format(std))
print(" - Sharpe ratio:\t {:.2f}".format((mean - rf)/ std))

###  Value weighted portfolios

In [ ]:
VW_data_mom = data_mom_b.copy()

VW_data_mom['VW_wL'] = (VW_data_mom['leg'] == -1) * VW_data_mom['mcap']
VW_data_mom['VW_wL_sum'] = VW_data_mom.groupby('date')['VW_wL'].transform('sum')
VW_data_mom['VW_wH'] = (VW_data_mom['leg'] == 1) * VW_data_mom['mcap']
VW_data_mom['VW_wH_sum'] = VW_data_mom.groupby('date')['VW_wH'].transform('sum')
VW_data_mom['VW_wL'] = VW_data_mom['VW_wL'] / VW_data_mom['VW_wL_sum']
VW_data_mom['VW_wH'] = VW_data_mom['VW_wH'] / VW_data_mom['VW_wH_sum']
VW_data_mom = VW_data_mom.drop(columns=['VW_wL_sum', 'VW_wH_sum'])
VW_data_mom['VW_w'] = VW_data_mom['VW_wL'] * VW_data_mom['leg'] + VW_data_mom['VW_wH'] * VW_data_mom['leg']
VW_data_mom['VW_ret'] = VW_data_mom['VW_w'] * VW_data_mom['ret']


# Create a dataframe that aggregates the returns, at each month and keep the risk free rate
VW_data_mom_ = VW_data_mom.groupby(['date']).agg({
    'VW_ret': 'sum', 
    RF_COL: 'first',
    }).reset_index()

# display(VW_data_mom_)

# Compute mean, std and Sharpe ratio
mean = VW_data_mom_['VW_ret'].mean() * 12
std = VW_data_mom_['VW_ret'].std() * np.sqrt(12)
rf = VW_data_mom_[RF_COL].mean() * 12

# Dispay the results
print(colored("Momentum strategy based on value weighted portfolios", "black", attrs=['underline', 'bold']))
print(" - Expected return:\t {:.2f}%".format(mean))
print(" - Standard deviation:\t {:.2f}%".format(std))
print(" - Sharpe ratio:\t {:.2f}".format((mean - rf)/ std))